In [1]:
%matplotlib inline
import os, glob
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import json
from skimage import data, img_as_float
from skimage import exposure
import PIL
from PIL import Image
import cv2
import random
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [2]:
annFile='../../annotations/instances_train2014.json'
coco=COCO(annFile)

loading annotations into memory...
Done (t=11.71s)
creating index...
index created!


In [3]:
#  Displaying coco categories and subcategories to choose from. Rather than choosing from one category we will choose combination of multiple categories.
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

COCO supercategories: 
kitchen animal appliance electronic food indoor outdoor sports furniture vehicle person accessory


In [4]:
# get all images containing given categories, select one at random
# person, surfboard -- Category combo 1
# person dog - Category combo 2
catIds = coco.getCatIds(catNms=['person','surfboard']);
imgIds1 = coco.getImgIds(catIds=catIds);
imgIds1 = imgIds1[:100]
catIds = coco.getCatIds(catNms=['person','dog']);
imgIds2 = coco.getImgIds(catIds=catIds);
imgIds2 = imgIds2[:100]
imgIds = imgIds1 + imgIds2
imgIds = list(dict.fromkeys(imgIds))
print(len(imgIds))

198


In [5]:
for x in imgIds:
    img = coco.loadImgs(x)[0]
    I = io.imread(img['coco_url'])
    path = os.path.join('../../dataCreate/', str(x))
    os.mkdir(path)
    io.imsave(os.path.join(path, '{}.png'.format(x)), I)

In [6]:
# load and display instance annotations
for x in imgIds:
    img = coco.loadImgs(x)[0]
    I = io.imread(img['coco_url'])
    plt.imshow(I); plt.axis('off')
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    imgA = coco.showAnns(anns)
    plt.savefig('../../dataCreate/{}/{}_Annotated'.format(x,x))
    plt.clf()

<Figure size 576x720 with 0 Axes>

In [7]:
# initialize COCO api for caption annotations
annFile = '../../annotations/captions_train2014.json'
coco_caps=COCO(annFile)

loading annotations into memory...
Done (t=0.80s)
creating index...
index created!


In [8]:
# load and display caption annotations
for x in imgIds:
    annIds = coco_caps.getAnnIds(imgIds=x);
    anns = coco_caps.loadAnns(annIds)
    with open('../../dataCreate/{}/{}.json'.format(x,x), 'w') as outfile:
        json.dump(anns, outfile)

In [9]:
def transposes(filename, dirpath):
    path = os.path.join(dirpath,filename)
    im = Image.open(path)
    hf = im.transpose(PIL.Image.FLIP_LEFT_RIGHT)
    hf.save("{}/hf_{}".format(dirpath, filename))
    vf = im.transpose(PIL.Image.FLIP_TOP_BOTTOM)
    vf.save("{}/vf_{}".format(dirpath, filename))
    r90 = im.transpose(PIL.Image.ROTATE_90)
    r90.save("{}/r90_{}".format(dirpath, filename))
    r180 = im.transpose(PIL.Image.ROTATE_180)
    r180.save("{}/r180_{}".format(dirpath, filename))
    r270 = im.transpose(PIL.Image.ROTATE_270)
    r270.save("{}/r270_{}".format(dirpath, filename))
    r45 = im.rotate(45, PIL.Image.NEAREST, expand = 1)
    r45.save("{}/r135_{}".format(dirpath, filename))
    r135 = im.rotate(135, PIL.Image.NEAREST, expand = 1)
    r135.save("{}/r135_{}".format(dirpath, filename))

In [10]:
def exposures(filename, dirpath):
    path = os.path.join(dirpath,filename)
    im = Image.open(path)
    img = np.asanyarray(im)

    # Contrast stretching
    p2, p98 = np.percentile(img, (2, 98))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))

    # Histogram Equalization
    img_eq = exposure.equalize_hist(img)

    # Adaptive Equalization
    img_adapteq = exposure.equalize_adapthist(img, clip_limit=0.03)

    #  Saving Images
    cv2.imwrite("{}/cvr_{}".format(dirpath, filename), img_rescale)
    im_re = Image.fromarray(img_rescale)
    im_re.save("{}/IR_{}".format(dirpath, filename))

    im_eq = Image.fromarray((img_eq * 255).astype(np.uint8))
    im_eq.save("{}/IE_{}".format(dirpath, filename))

    im_ad = Image.fromarray((img_adapteq * 255).astype(np.uint8))
    im_ad.save("{}/IA_{}".format(dirpath, filename))

    cv2.imwrite("{}/cve_{}".format(dirpath, filename), (img_eq * 255).astype(np.uint8))

    cv2.imwrite("{}/cva_{}".format(dirpath, filename), (img_adapteq * 255).astype(np.uint8))


In [11]:
import pathlib
pathlib.Path().absolute()

PosixPath('/Users/manvishn/refcoco/cocoapi/PythonAPI')

In [12]:
from os import walk
directory = "/Users/manvishn/refcoco/dataCreate"
for (dirpath, dirnames, filenames) in walk(directory):
    for f in filenames:
         if f.endswith('.png'):
            exposures(f, dirpath)
            transposes(f, dirpath)

In [13]:
dataset =[]

In [14]:
for (dirpath, dirnames, filenames) in walk(directory):
    for f in filenames:
         if f.endswith('.png') and 'Annotated' not in f:
             with open(os.path.join(dirpath,dirpath.split('/')[-1]+'.json')) as json_file:
                data = json.load(json_file)
             refs = []
             for x in data:
                 refs.append(x['caption'])
             refs = random.sample(refs,3)
             dict = {
                 "filename":f,
                 "segmented_filename": f.split('.')[0]+'_Annotated.png',
                 "references": refs
             }
             dataset.append(dict)

In [15]:
final_json = {
    "Name": "Aradhana Trivedi",
    "ASUID": 1217102484,
    "images":dataset
}

In [16]:
with open('data_json.json', 'w') as outfile:
    json.dump(final_json, outfile)